## Python Requests Library

### Basic Usage
#### GET
- requests.get(url)
- requests.get(url, headers=headers)
- requests.get(url, params=params)

#### POST
- requests.post(url)
- requests.post(url, data=params)

#### PUT
- requests.put(url)

#### DELETE
- requests.delete(url)

#### Session
<pre>
s = requests.Session()
s.get(url)
...
</pre>

### Example: Gitbucket
API-ref : https://github.com/gitbucket/gitbucket/wiki/API-WebHook

In [1]:
from IPython.display import display
import json
import requests

GITBUCKET_API_TOKEN="f34d5ff4b46717aabc8192223e3be647d689286b"
BASE_URL="http://127.0.0.1:8080/api/v3"
OWNER="root"
REPO="testrepo"

headers = {
    'Content-Type': 'application/json',
    'Authorization': 'token %s' % GITBUCKET_API_TOKEN
}

/usr/local/lib/python2.7/dist-packages/requests/__init__.py:83: RequestsDependencyWarning: Old version of cryptography ([1, 2, 3]) may cause slowdown.
  warnings.warn(warning, RequestsDependencyWarning)


In [2]:
# List Pullrequests
# GET /repos/:owner/:repo/pulls
url = "%s/repos/%s/%s/pulls" % (BASE_URL, OWNER, REPO)

r = requests.get(url, headers=headers)
for pull_request_data in r.json():
    print(pull_request_data["title"], pull_request_data["number"])

(u'test branch 2nd', 2)
(u'test branch', 1)


In [3]:
# Create Pullrequest
# POST /repos/:owner/:repo/pulls
url = "%s/repos/%s/%s/pulls" % (BASE_URL, OWNER, REPO)
new_branch_name = "new_feature"

params = {
  "title": "New feature",
  "body": "This is new feature",
  "head": new_branch_name,
  "base": "master",
}

r = requests.post(url, headers=headers, data=json.dumps(params))
r

<Response [200]>

In [4]:
# Create Comment
# POST /repos/:owner/:repo/issues/:number/comments
pull_request_no = 1
url = "%s/repos/%s/%s/issues/%s/comments" % (BASE_URL, OWNER, REPO, pull_request_no)

params= {
  "body": "LGTM!!"
}

r = requests.post(url, headers=headers, data=json.dumps(params))
r

<Response [200]>

### Example: Media Wiki
API-ref: https://www.mediawiki.org/wiki/API:Main_page

In [5]:
BASE_URL="http://127.0.0.1/api.php"
USERNAME="test"
PASSWORD="testtest"

In [6]:
# Login
# Get a login token
#   GET: api.php?action=query&meta=tokens&type=login
# Login
#   POST: api.php?action=login&lgname=user&lgpassword=password&lgtoken=<token>
session = requests.Session()

url = BASE_URL
params = {
    'format': "json",
    'action': "query",
    'meta': "tokens",
    'type': "login",
}

r = session.get(url, params=params)
r.json()

{u'batchcomplete': u'',
 u'query': {u'tokens': {u'logintoken': u'2f093b2423fb013e7204635049e0b9df5c55737c+\\'}}}

In [7]:
token = r.json()["query"]["tokens"]["logintoken"]

params = {
    'format': "json",
    'action': "login",
    'lgname': USERNAME,
    'lgpassword': PASSWORD,
    'lgtoken': token,
}

r = session.post(url, data=params)
r.json()

{u'login': {u'lguserid': 1, u'lgusername': u'Test', u'result': u'Success'},
 u'warnings': {u'login': {u'*': u'Main-account login via "action=login" is deprecated and may stop working without warning. To continue login with "action=login", see [[Special:BotPasswords]]. To safely continue using main-account login, see "action=clientlogin".'},
  u'main': {u'*': u'Subscribe to the mediawiki-api-announce mailing list at <https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce> for notice of API deprecations and breaking changes.'}}}

In [8]:
# Read a page for only logged-in user.
page_titles = ["Main_Page"]
params = {
    'format': 'json',
    'action': 'query',
    'titles': page_titles,
    'prop': 'revisions',
    'rvprop': 'content',
}

data = session.get(url, params=params)

texts = []
for _, page in data.json()['query']['pages'].items():
    texts = page['revisions'][0]['*']

if texts:
    for text in texts.split('\n'):
        print(text)

test test
test test
